# APIM ❤️ OpenAI

## メッセージ保存ラボ
![flow](../../images/message-storing.gif)

[イベントハブへのログ](https://learn.microsoft.com/en-us/azure/api-management/log-to-eventhub-policy)ポリシーを通じて、メッセージの詳細をCosmos DBに保存するテスト用プレイグラウンド。このポリシーを使用すると、DBに保存するデータ（モデル、リージョン、プロンプト、完了、トークン、安全フィルターなど）を簡単に制御できます。

### 結果
![result](result.png)

### 目次
- [0️⃣ ノートブック変数の初期化](#0)
- [1️⃣ Azureリソースグループの作成](#1)
- [2️⃣ 🦾 Bicepを使用したデプロイの作成](#2)
- [3️⃣ デプロイの出力を取得](#3)
- [🧪 直接HTTPコールを使用したAPIのテスト](#requests)
- [🧪 Azure OpenAI Python SDKを使用したAPIのテスト](#sdk)
- [🔍 CosmosDBアイテムの分析](#cosmosdb)
- [📊 インサイトの抽出とデータの可視化](#fabric)
- [🗑️ リソースのクリーンアップ](#clean)

### 前提条件
- [Python 3.8以降のバージョン](https://www.python.org/)がインストールされていること
- [Pandasライブラリ](https://pandas.pydata.org/)とazure-cosmosライブラリがインストールされていること
- [VS Code](https://code.visualstudio.com/)がインストールされ、[Jupyterノートブック拡張機能](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)が有効になっていること
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli)がインストールされていること
- Contributor権限を持つ[Azureサブスクリプション](https://azure.microsoft.com/en-us/free/)があること
- [Azure OpenAIへのアクセス](https://aka.ms/oai/access)が許可されていること、またはモックサービスを有効にすること
- [Azure CLIでAzureにサインイン](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)していること

<a id='0'></a>
### 0️⃣ ノートブック変数の初期化

- リソースはサブスクリプションIDに基づいた一意の文字列でサフィックスされます
- ```mock_webapps``` 変数は、モック機能のためにデプロイされたWebアプリのリストを設定します。モックサービスでOpenAIの動作をシミュレートするために、```openai_resources``` リストをクリアします。
- ロケーションパラメータは、[Azureリージョンごとの製品の利用可能性](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)に基づいて調整してください。
- OpenAIモデルとバージョンは、[リージョンごとの利用可能性](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)に基づいて調整してください。

In [ ]:
import os
import json
import datetime
import requests

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"
apim_resource_name = "apim"
apim_resource_location = "westeurope"
apim_resource_sku = "Basicv2"
openai_resources = [ {"name": "openai1", "location": "swedencentral"}, {"name": "openai2", "location": "francecentral"} ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources
openai_resources_sku = "S0"
openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ {"name": "openaimock1", "endpoint": "https://openaimock1.azurewebsites.net"}, {"name": "openaimock2", "endpoint": "https://openaimock2.azurewebsites.net"} ]

log_analytics_name = "workspace"
app_insights_name = 'insights'

eventhub_namespace_name = "eventhub"
eventhub_logger_name = "eventhub-logger"
eventhub_name = "openai-messages"
streamingjobs_name = "streamingjobs"
cosmosdb_account_name = "cosmosdb"
cosmosdb_database_name = "openaidb"
cosmosdb_container_name = "messages"


<a id='1'></a>
### 1️⃣ Azureリソースグループの作成
このラボでデプロイされるすべてのリソースは、指定されたリソースグループに作成されます。既存のリソースグループを使用する場合は、このステップをスキップしてください。

In [ ]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

<a id='2'></a>
### 2️⃣ 🦾 Bicepを使用したデプロイの作成

このラボでは、[Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep)を使用して、デプロイされるすべてのリソースを宣言的に定義します。異なる構成を試すために、パラメータや[main.bicep](main.bicep)を直接変更してください。

In [ ]:
if len(openai_resources) > 0:
    backend_id = openai_backend_pool if len(openai_resources) > 1 else openai_resources[0].get("name")
elif len(mock_webapps) > 0:
    backend_id = mock_backend_pool if len(mock_backend_pool) > 1 else mock_webapps[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id}", backend_id).replace("{eventhub-logger-name}", eventhub_logger_name)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name },
    "eventHubNamespaceName": { "value": eventhub_namespace_name },
    "eventHubLoggerName": { "value": eventhub_logger_name },
    "eventHubName": { "value": eventhub_name },
    "streamingJobsName": { "value": streamingjobs_name },
    "cosmosDBAccountName": { "value": cosmosdb_account_name },
    "cosmosDBDatabaseName": { "value": cosmosdb_database_name },
    "cosmosDBContainerName": { "value": cosmosdb_container_name }
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)


<a id='3'></a>
### 3️⃣ デプロイの出力を取得

テストの準備が整う前に、ゲートウェイURLとサブスクリプションを取得する必要があります。

In [ ]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = deployment_stdout.n
print("👉🏻 Workspace ID: ", workspace_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = deployment_stdout.n
print("👉🏻 App ID: ", app_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.cosmosDBConnectionString.value -o tsv
cosmosdb_connection_string = deployment_stdout.n



<a id='requests'></a>
### 🧪 直接HTTPコールを使用したAPIのテスト
Requestsは、ここで生のAPIリクエストを行い、レスポンスを検査するために使用される、エレガントでシンプルなPython用のHTTPライブラリです。

In [ ]:
import time
runs = 2
sleep_time_ms = 1000

url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version

for i in range(runs):
    print("▶️ Run: ", i+1)
    if len(openai_resources) > 0:
        messages={"messages":[
            {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
            {"role": "user", "content": "Can you tell me the time, please?"}
        ]}
    elif len(mock_webapps) > 0:
        messages={
            "messages": [
                {
                    "role": "system", 
                    "content": {
                        "simulation": {
                            "default": {"response_status_code": 200, "wait_time_ms": 0},
                            "openaimock1.azurewebsites.net": {"response_status_code": 429}
                        }
                    }
                }
            ]
        }
    response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)
    print("status code: ", response.status_code)
    print("headers ", response.headers)
    print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request
    if (response.status_code == 200):
        data = json.loads(response.text)
        # print(json.dumps(data, indent=4))
        print("response: ", data.get("choices")[0].get("message").get("content"))
    else:
        print(response.text)
    time.sleep(sleep_time_ms/1000)


<a id='sdk'></a>
### 🧪 Azure OpenAI Python SDKを使用したAPIのテスト
OpenAPIは広く使用されている[Pythonライブラリ](https://github.com/openai/openai-python)を提供しています。このライブラリには、すべてのリクエストパラメータとレスポンスフィールドの型定義が含まれています。このテストの目的は、APIMがOpenAIへのリクエストをシームレスにプロキシし、その機能を妨げることなく動作することを確認することです。
- 注意: このステップを実行する前に、ターミナルで```pip install openai```を実行してください。

In [ ]:
import time
runs = 2
sleep_time_ms = 1000

for i in range(runs):
    print("▶️ Run: ", i+1)
    from openai import AzureOpenAI
    if len(openai_resources) > 0:
        messages=[
            {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
            {"role": "user", "content": "Can you tell me the time, please?"}
        ]
    elif len(mock_webapps) > 0:
        messages=[
                {
                    "role": "system", 
                    "content": {
                        "simulation": {
                            "default": {"response_status_code": 200, "wait_time_ms": 0},
                            "openaimock1.azurewebsites.net": {"response_status_code": 429}
                        }
                    }
                }
            ]
    client = AzureOpenAI(
        azure_endpoint=apim_resource_gateway_url,
        api_key=apim_subscription_key,
        api_version=openai_api_version
    )
    response = client.chat.completions.create(model=openai_model_name, messages=messages)
    print(response.choices[0].message.content)
    time.sleep(sleep_time_ms/1000)


<a id='cosmosdb'></a>
### 🔍 CosmosDBアイテムの分析

以下のスクリプトを実行する前に、`pip install azure-cosmos`を実行してください。

In [ ]:
import pandas as pd
from azure.core.exceptions import AzureError
from azure.cosmos import CosmosClient, PartitionKey

client = CosmosClient.from_connection_string(conn_str=cosmosdb_connection_string)
database = client.get_database_client(cosmosdb_database_name)
container = database.get_container_client(cosmosdb_container_name)

item_list = list(container.read_all_items(max_item_count=10))

print('Found {0} items'.format(item_list.__len__()))

df = pd.DataFrame(container.query_items('select c.timestamp, c.model, c.modelRegion, c.apimRequestId, c.clientIp, c.totalTokens, c.request, c.response from c',enable_cross_partition_query=True))
df


<a id='fabric'></a>
### 📊 インサイトの抽出とデータの可視化

![リアルタイムインテリジェンス](https://learn.microsoft.com/en-us/fabric/real-time-intelligence/media/overview/overview-schematic.png)

- Event Hubにデータがあるので、Microsoft Fabricの[Eventstream](https://learn.microsoft.com/en-us/fabric/real-time-intelligence/event-streams/add-source-azure-event-hubs?pivots=enhanced-capabilities)を作成し、Event Hubに接続して、OpenAI APIコールを取り込み、処理、分析、変換し、[リアルタイムインテリジェンス](https://learn.microsoft.com/en-us/fabric/real-time-intelligence/overview)を使用してアクションを実行できます。
- また、[Cosmos DBミラーリング](https://learn.microsoft.com/en-us/fabric/database/mirrored-database/azure-cosmos-db)を使用して、Cosmos DBデータをほぼリアルタイムでFabric OneLakeに継続的にレプリケートすることもできます。

<a id='clean'></a>
### 🗑️ リソースのクリーンアップ

ラボが終了したら、追加の料金を避け、Azureサブスクリプションを整理するために、デプロイしたすべてのリソースをAzureから削除する必要があります。
そのためには、[リソースのクリーンアップノートブック](clean-up-resources.ipynb)を使用してください。